###Import packages and read datasets(If you need data, please email haoyan.jiang@mail.utoronto.ca for data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from statistics import mean, variance, mode
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_e = pd.read_csv('Your_Eyemovement_Data.csv')
df_e.head()

,Unnamed: 0,Recording timestamp,Gaze point X,Gaze point Y,Gaze point 3D X,Gaze point 3D Y,Gaze point 3D Z,Gaze direction left X,Gaze direction left Y,Gaze direction left Z,...,Eye movement type,Gaze event duration,Eye movement type index,Fixation point X,Fixation point Y,interest_level,order,videoNum,videoSeq,is_distraction
0,0,1.019391,662.0,1022.0,262.7431,-420.3356,972.3964,0.23289,-0.35110,0.90691,...,Fixation,160.0,5.0,662.0,999.0,NaN,3.0,1.0,1.0,NaN
1,1,1.039377,665.0,1042.0,259.6674,-434.7281,967.5176,0.24141,-0.35704,0.90235,...,Fixation,160.0,5.0,662.0,999.0,NaN,3.0,1.0,1.0,NaN
2,2,1.059358,652.0,996.0,792.1335,-1162.3524,2837.9634,0.24656,-0.35477,0.90186,...,Fixation,160.0,5.0,662.0,999.0,NaN,3.0,1.0,1.0,NaN
3,3,1.079347,645.0,994.0,756.1740,-1079.8293,2647.4790,0.25044,-0.35167,0.90200,...,Fixation,160.0,5.0,662.0,999.0,NaN,3.0,1.0,1.0,NaN
4,4,1.099322,647.0,1010.0,281.1833,-417.7545,990.3439,0.23809,-0.34010,0.90975,...,Fixation,160.0,5.0,662.0,999.0,NaN,3.0,1.0,1.0,NaN


In [ ]:
df_a = pd.read_csv('Your_Accelerometer_Data.csv')
df_a.head()

,Unnamed: 0,Recording timestamp,Eye movement type,Gaze event duration,Eye movement type index,Fixation point X,Fixation point Y,Accelerometer X,Accelerometer Y,Accelerometer Z,interest_level,order,videoNum,videoSeq,is_distraction
0,0,1.006490,Saccade,40.0,5.0,NaN,NaN,-0.1569,-8.3310,-4.2572,NaN,3.0,1.0,1.0,NaN
1,1,1.016187,Fixation,160.0,5.0,662.0,999.0,-0.2746,-8.2537,-4.2436,NaN,3.0,1.0,1.0,NaN
2,2,1.025884,Fixation,160.0,5.0,662.0,999.0,-0.3923,-8.2083,-4.2754,NaN,3.0,1.0,1.0,NaN
3,3,1.035581,Fixation,160.0,5.0,662.0,999.0,-0.5099,-8.1696,-4.2686,NaN,3.0,1.0,1.0,NaN
4,4,1.045278,Fixation,160.0,5.0,662.0,999.0,-0.5884,-8.2083,-4.2754,NaN,3.0,1.0,1.0,NaN


In [ ]:
df_g = pd.read_csv('Your_Gyro_Data.csv')
df_g.head()

,Unnamed: 0,Recording timestamp,Eye movement type,Gaze event duration,Eye movement type index,Fixation point X,Fixation point Y,Gyro X,Gyro Y,Gyro Z,interest_level,order,videoNum,videoSeq,is_distraction
0,0,1.007987,Saccade,40.0,5.0,NaN,NaN,-8.40,-6.4879,0.9173,NaN,3.0,1.0,1.0,NaN
1,1,1.018566,Fixation,160.0,5.0,662.0,999.0,-5.53,-5.5552,0.8685,NaN,3.0,1.0,1.0,NaN
2,2,1.029145,Fixation,160.0,5.0,662.0,999.0,-2.38,-3.5357,1.5089,NaN,3.0,1.0,1.0,NaN
3,3,1.039724,Fixation,160.0,5.0,662.0,999.0,1.47,-1.2487,1.8411,NaN,3.0,1.0,1.0,NaN
4,4,1.050303,Fixation,160.0,5.0,662.0,999.0,5.67,1.8290,1.3176,NaN,3.0,1.0,1.0,NaN


##Define functions for preprocessing and aggregation for original data

In [ ]:
def distract_att(df1, window_l):
  a = 0
  temp_list = []
  type_list_a = []
  for i in range(len(df1['Recording timestamp'])):
    if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
          if df1['is_distraction'][i] == True:
            temp_list.append('True')
          elif df1['is_distraction'][i] == False:
            temp_list.append('False')
          else:
            temp_list.append('None')
          #temp_list.append(df1["is_distraction"][i])
    elif df1['Recording timestamp'][i] >= window_l[a][1]:
      type_list_a.append(temp_list)
      a += 1
      if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
        temp_list = []
        if df1['Recording timestamp'][i] == True:
          temp_list.append('True')
        elif df1['Recording timestamp'][i] == False:
          temp_list.append('False')
        else:
          temp_list.append('None')
        if window_l[a] == window_l[-1]:
          type_list_a.append(temp_list)
    if len(window_l) == 1:
      if i == len(df1['Recording timestamp']) - 1:
        type_list_a.append(temp_list)
  return type_list_a

In [ ]:
def process_lists(df1, df2, df3, window_l):
    da1  = distract_att(df1, window_l)
    da2  = distract_att(df2, window_l)
    da3  = distract_att(df3, window_l)
    a5 = []
    for i in range(len(a1)):
        a4 = da1[i] + da2[i] + da3[i]
        a5.append(a4)
        a4 = []
    a6 = []
    for i in a5:
        if 'True' in i and 'False' not in i:
            a6.append('True')
        elif 'False' in i and 'True' not in i:
            a6.append('False')
        elif 'False' in i and 'True' in i:
            a6.append('True')
        else:
            a6.append('None')
    return a6


In [ ]:
def outlier_function(df, x):
  v_no = df.loc[df['videoNum'] == x]
  v_no = v_no.reset_index(drop=True)
  for i in range(len(v_no['Recording timestamp'])):
    if i < len(v_no['Recording timestamp']) and abs(v_no['Recording timestamp'][i+1] - v_no['Recording timestamp'][i]) > 1:
      return v_no.head(i+1), v_no.tail(len(v_no['Recording timestamp']) - i - 1 ).reset_index(drop=True)

In [ ]:
def win_list(x, y, z):
  start = x
  end = y

# Define the interval
  interval = z

# Create a list using a list comprehension
  my_list = [round(start + i * interval, 2) for i in range(int((end - start) / interval) + 1)]
  window_list = []
  list_1 = []
  for i in range(len(my_list)):
    if my_list[i] < my_list[-1]:
      list_1.append(my_list[i])
      list_1.append(my_list[i+1])
      window_list.append(list_1)
      list_1 = []
  return window_list

In [ ]:
def eye_move_type_list(df1, df2, df3, window_l):
    a_index = 0
    temp_list = []
    type_list_a = []

    for i in range(len(df1['Recording timestamp'])):
        if window_l[a_index][0] <= df1['Recording timestamp'][i] < window_l[a_index][1]:
            temp_list.append(df1["Eye movement type"][i])
        elif df1['Recording timestamp'][i] >= window_l[a_index][1]:
            type_list_a.append(temp_list)
            a_index += 1
            if window_l[a_index][0] <= df1['Recording timestamp'][i] < window_l[a_index][1]:
                temp_list = []
                temp_list.append(df1["Eye movement type"][i])
                if window_l[a_index] == window_l[-1]:
                    type_list_a.append(temp_list)
        if len(window_l) == 1:
            if i == len(df1['Recording timestamp']) - 1:
                type_list_a.append(temp_list)

    b_index = 0
    temp_type_list_1 = []
    type_list_b = []

    for i in range(len(df2['Recording timestamp'])):
        if window_l[b_index][0] <= df2['Recording timestamp'][i] < window_l[b_index][1]:
            temp_type_list_1.append(df2["Eye movement type"][i])
        elif df2['Recording timestamp'][i] >= window_l[b_index][1]:
            type_list_b.append(temp_type_list_1)
            b_index += 1
            if window_l[b_index][0] <= df2['Recording timestamp'][i] < window_l[b_index][1]:
                temp_type_list_1 = []
                temp_type_list_1.append(df2["Eye movement type"][i])
                if window_l[b_index] == window_l[-1]:
                    type_list_b.append(temp_type_list_1)
        if len(window_l) == 1:
            if i == len(df2['Recording timestamp']) - 1:
                type_list_b.append(temp_type_list_1)

    c_index = 0
    temp_type_list_2 = []
    type_list_c = []

    for i in range(len(df3['Recording timestamp'])):
        if window_l[c_index][0] <= df3['Recording timestamp'][i] < window_l[c_index][1]:
            temp_type_list_2.append(df3["Eye movement type"][i])
        elif df3['Recording timestamp'][i] >= window_l[c_index][1]:
            type_list_c.append(temp_type_list_2)
            c_index += 1
            if window_l[c_index][0] <= df3['Recording timestamp'][i] < window_l[c_index][1]:
                temp_type_list_2 = []
                temp_type_list_2.append(df3["Eye movement type"][i])
                if window_l[c_index] == window_l[-1]:
                    type_list_c.append(temp_type_list_2)
        if len(window_l) == 1:
            if i == len(df3['Recording timestamp']) - 1:
                type_list_c.append(temp_type_list_2)

    temp_type_list = []
    elements_to_check = ['Saccade', 'Fixation', 'Unclassified', 'EyesNotFound']
    type_list = []

    for i in range(len(type_list_a)):
        temp_type_list.append(type_list_a[i] + type_list_b[i] + type_list_c[i])

    for i in range(len(temp_type_list)):
        if temp_type_list[i].count('Saccade') != 0 or temp_type_list[i].count('Fixation') != 0:
            if temp_type_list[i].count('Saccade') >= temp_type_list[i].count('Fixation'):
                type_list.append('Saccade')
            elif temp_type_list[i].count('Saccade') < temp_type_list[i].count('Fixation'):
                type_list.append('Fixation')
        else:
            if temp_type_list[i].count('Unclassified') != 0:
                type_list.append('Unclassified')
            else:
                type_list.append('EyesNotFound')

    return type_list


In [ ]:
def new_column_list(df1, df2, df3, col_name, window_l):
    a = 0
    temp_list = []
    type_list_a = []
    for i in range(len(df1['Recording timestamp'])):
        if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
            if not np.isnan(df1[col_name][i]):
                temp_list.append(df1[col_name][i])
        elif df1['Recording timestamp'][i] >= window_l[a][1]:
            type_list_a.append(temp_list)
            a += 1
            if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
                temp_list = []
                if not np.isnan(df1[col_name][i]):
                    temp_list.append(df1[col_name][i])
                if window_l[a] == window_l[-1]:
                    type_list_a.append(temp_list)
    if len(window_l) == 1:
      if i == len(df1['Recording timestamp']) - 1:
        type_list_a.append(temp_list)

    b = 0
    temp_type_list_1 = []
    type_list_e = []
    for i in range(len(df2['Recording timestamp'])):
        if window_l[b][0] <= df2['Recording timestamp'][i] < window_l[b][1]:
            if not np.isnan(df2[col_name][i]):
                temp_type_list_1.append(df2[col_name][i])
        elif df2['Recording timestamp'][i] >= window_l[b][1]:
            type_list_e.append(temp_type_list_1)
            b += 1
            if window_l[b][0] <= df2['Recording timestamp'][i] < window_l[b][1]:
                temp_type_list_1 = []
                if not np.isnan(df2[col_name][i]):
                    temp_type_list_1.append(df2[col_name][i])
                if window_l[b] == window_l[-1]:
                    type_list_e.append(temp_type_list_1)
    if len(window_l) == 1:
      if i == len(df2['Recording timestamp']) - 1:
        type_list_e.append(temp_type_list_1)

    c = 0
    temp_type_list_2 = []
    type_list_g = []
    for i in range(len(df3['Recording timestamp'])):
        if window_l[c][0] <= df3['Recording timestamp'][i] < window_l[c][1]:
            if not np.isnan(df3[col_name][i]):
                temp_type_list_2.append(df3[col_name][i])
        elif df3['Recording timestamp'][i] >= window_l[c][1]:
            type_list_g.append(temp_type_list_2)
            c += 1
            if window_l[c][0] <= df3['Recording timestamp'][i] < window_l[c][1]:
                temp_type_list_2 = []
                if not np.isnan(df3[col_name][i]):
                    temp_type_list_2.append(df3[col_name][i])
                if window_l[c] == window_l[-1]:
                    type_list_g.append(temp_type_list_2)
    if len(window_l) == 1:
      if i == len(df3['Recording timestamp']) - 1:
        type_list_g.append(temp_type_list_2)

    final_temp_list = []
    final_list = []
    for i in range(len(type_list_a)):
        final_temp_list.append(type_list_a[i] + type_list_e[i] + type_list_g[i])
    for i in range(len(final_temp_list)):
        final_list.append(np.mean(final_temp_list[i]))

    return final_list


In [ ]:
def new_column_list(df1, df2, df3, col_name, window_l):
    a_index = 0
    temp_list = []
    type_list_a = []

    for i in range(len(df1['Recording timestamp'])):
        timestamp = df1['Recording timestamp'][i]

        if window_l[a_index][0] <= timestamp < window_l[a_index][1]:
            if not np.isnan(df1[col_name][i]):
                temp_list.append(df1[col_name][i])
        elif timestamp >= window_l[a_index][1]:
            type_list_a.append(temp_list)
            a_index += 1

            if window_l[a_index][0] <= timestamp < window_l[a_index][1]:
                temp_list = []
                if not np.isnan(df1[col_name][i]):
                    temp_list.append(df1[col_name][i])

                if window_l[a_index] == window_l[-1]:
                    type_list_a.append(temp_list)

        if len(window_l) == 1:
            if i == len(df1['Recording timestamp']) - 1:
                type_list_a.append(temp_list)

    b_index = 0
    temp_type_list_1 = []
    type_list_b = []

    for i in range(len(df2['Recording timestamp'])):
        timestamp = df2['Recording timestamp'][i]

        if window_l[b_index][0] <= timestamp < window_l[b_index][1]:
            if not np.isnan(df2[col_name][i]):
                temp_type_list_1.append(df2[col_name][i])
        elif timestamp >= window_l[b_index][1]:
            type_list_b.append(temp_type_list_1)
            b_index += 1

            if window_l[b_index][0] <= timestamp < window_l[b_index][1]:
                temp_type_list_1 = []
                if not np.isnan(df2[col_name][i]):
                    temp_type_list_1.append(df2[col_name][i])

                if window_l[b_index] == window_l[-1]:
                    type_list_b.append(temp_type_list_1)

        if len(window_l) == 1:
            if i == len(df2['Recording timestamp']) - 1:
                type_list_b.append(temp_type_list_1)

    c_index = 0
    temp_type_list_2 = []
    type_list_c = []

    for i in range(len(df3['Recording timestamp'])):
        timestamp = df3['Recording timestamp'][i]

        if window_l[c_index][0] <= timestamp < window_l[c_index][1]:
            if not np.isnan(df3[col_name][i]):
                temp_type_list_2.append(df3[col_name][i])
        elif timestamp >= window_l[c_index][1]:
            type_list_c.append(temp_type_list_2)
            c_index += 1

            if window_l[c_index][0] <= timestamp < window_l[c_index][1]:
                temp_type_list_2 = []
                if not np.isnan(df3[col_name][i]):
                    temp_type_list_2.append(df3[col_name][i])

                if window_l[c_index] == window_l[-1]:
                    type_list_c.append(temp_type_list_2)

        if len(window_l) == 1:
            if i == len(df3['Recording timestamp']) - 1:
                type_list_c.append(temp_type_list_2)

    final_temp_list = []
    final_list = []

    for i in range(len(type_list_a)):
        final_temp_list.append(type_list_a[i] + type_list_b[i] + type_list_c[i])

    for i in range(len(final_temp_list)):
        final_list.append(np.mean(final_temp_list[i]))

    return final_list


In [ ]:
def new_unique_column_list(df1, col_name, window_l):
    a = 0
    temp_list = []
    type_list_a = []
    for i in range(len(df1['Recording timestamp'])):
        if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
            if not np.isnan(df1[col_name][i]):
                temp_list.append(df1[col_name][i])
        elif df1['Recording timestamp'][i] >= window_l[a][1]:
            type_list_a.append(temp_list)
            a += 1
            if window_l[a][0] <= df1['Recording timestamp'][i] < window_l[a][1]:
                temp_list = []
                if not np.isnan(df1[col_name][i]):
                    temp_list.append(df1[col_name][i])
                if window_l[a] == window_l[-1]:
                    type_list_a.append(temp_list)
    if len(window_l) == 1:
      if i == len(df1['Recording timestamp']) - 1:
        type_list_a.append(temp_list)
    final_list = []
    for i in range(len(type_list_a)):
        final_list.append(np.nanmean(type_list_a[i]))
    return final_list

Finish aggregate and preprocess the original data

In [ ]:
def output_data(dfg1, dfe1, dfa1, window_list):
    new_gaze_event_duration = new_column_list(dfg1, dfe1, dfa1, 'Gaze event duration', window_list)
    new_Eye_movement_type_index = new_column_list(dfg1, dfe1, dfa1, "Eye movement type index", window_list)
    new_Fixation_point_X = new_column_list(dfg1, dfe1, dfa1, "Fixation point X", window_list)
    new_Fixation_point_Y = new_column_list(dfg1, dfe1, dfa1, "Fixation point Y", window_list)
    new_Eye_movement_type = eye_move_type_list(dfg1, dfe1, dfa1, window_list)
    new_interest_level = new_column_list(dfg1, dfe1, dfa1, "interest_level", window_list)


    new_Accelerometer_X = new_unique_column_list(dfa1, "Accelerometer X", window_list)
    new_Accelerometer_Y = new_unique_column_list(dfa1, "Accelerometer Y", window_list)
    new_Accelerometer_Z = new_unique_column_list(dfa1, "Accelerometer Z", window_list)


    new_Gyro_X = new_unique_column_list(dfg1, "Gyro X", window_list)
    new_Gyro_Y = new_unique_column_list(dfg1, "Gyro Y", window_list)
    new_Gyro_Z = new_unique_column_list(dfg1, "Gyro Z", window_list)


    new_Gaze_point_X = new_unique_column_list(dfe1, "Gaze point X", window_list)
    new_Gaze_point_Y = new_unique_column_list(dfe1, "Gaze point Y", window_list)

    new_Gaze_point_3D_X = new_unique_column_list(dfe1, "Gaze point 3D X", window_list)
    new_Gaze_point_3D_Y = new_unique_column_list(dfe1, "Gaze point 3D Y", window_list)
    new_Gaze_point_3D_Z = new_unique_column_list(dfe1, "Gaze point 3D Z", window_list)

    new_Gaze_direction_left_X = new_unique_column_list(dfe1, "Gaze direction left X", window_list)
    new_Gaze_direction_left_Y = new_unique_column_list(dfe1, "Gaze direction left Y", window_list)
    new_Gaze_direction_left_Z = new_unique_column_list(dfe1, "Gaze direction left Z", window_list)

    new_Gaze_direction_right_X = new_unique_column_list(dfe1, "Gaze direction right X", window_list)
    new_Gaze_direction_right_Y = new_unique_column_list(dfe1, "Gaze direction right Y", window_list)
    new_Gaze_direction_right_Z = new_unique_column_list(dfe1, "Gaze direction right Z", window_list)

    new_Pupil_position_left_X = new_unique_column_list(dfe1, "Pupil position left X", window_list)
    new_Pupil_position_left_Y = new_unique_column_list(dfe1, "Pupil position left Y", window_list)
    new_Pupil_position_left_Z = new_unique_column_list(dfe1, "Pupil position left Z", window_list)

    new_Pupil_position_right_X = new_unique_column_list(dfe1, "Pupil position right X", window_list)
    new_Pupil_position_right_Y = new_unique_column_list(dfe1, "Pupil position right Y", window_list)
    new_Pupil_position_right_Z = new_unique_column_list(dfe1, "Pupil position right Z", window_list)

    new_Pupil_diameter_left = new_unique_column_list(dfe1, "Pupil diameter left", window_list)
    new_Pupil_diameter_right = new_unique_column_list(dfe1, "Pupil diameter right", window_list)

    new_Gaze_event_duration = new_unique_column_list(dfe1, "Gaze event duration", window_list)
    new_process_lists = process_lists(dfg1, dfe1, dfa1, window_list)

    data = {
    "Gaze event duration": new_gaze_event_duration,
    "Eye movement type index": new_Eye_movement_type_index,
    "Fixation point X": new_Fixation_point_X,
    "Fixation point Y": new_Fixation_point_Y,
    "Eye movement type": new_Eye_movement_type,
    "Accelerometer X": new_Accelerometer_X,
    "Accelerometer Y": new_Accelerometer_Y,
    "Accelerometer Z": new_Accelerometer_Z,
    "Gyro X": new_Gyro_X,
    "Gyro Y": new_Gyro_Y,
    "Gyro Z": new_Gyro_Z,
    "Gaze point X": new_Gaze_point_X,
    "Gaze point Y": new_Gaze_point_Y,
    "Gaze point 3D X": new_Gaze_point_3D_X,
    "Gaze point 3D Y": new_Gaze_point_3D_Y,
    "Gaze point 3D Z": new_Gaze_point_3D_Z,
    "Gaze direction left X": new_Gaze_direction_left_X,
    "Gaze direction left Y": new_Gaze_direction_left_Y,
    "Gaze direction left Z": new_Gaze_direction_left_Z,
    "Gaze direction right X": new_Gaze_direction_right_X,
    "Gaze direction right Y": new_Gaze_direction_right_Y,
    "Gaze direction right Z": new_Gaze_direction_right_Z,
    "Pupil position left X": new_Pupil_position_left_X,
    "Pupil position left Y": new_Pupil_position_left_Y,
    "Pupil position left Z": new_Pupil_position_left_Z,
    "Pupil position right X": new_Pupil_position_right_X,
    "Pupil position right Y": new_Pupil_position_right_Y,
    "Pupil position right Z": new_Pupil_position_right_Z,
    "Pupil diameter left": new_Pupil_diameter_left,
    "Pupil diameter right": new_Pupil_diameter_right,
    "interest_level" :new_interest_level,
    "is_distraction": new_process_lists}

# create the DataFrame
    df_v1 = pd.DataFrame(data)
# print the first 5 rows of the DataFrame
    return df_v1


Preprocess the rest useful columns

In [ ]:
def other_columns(df, order_num, video_num, video_seq, up_win_l, low_win_l):
  df['order'] = [order_num] * len(df['Gyro X'])
  df['videoNum'] = [video_num] * len(df['Gyro X'])
  df['videoSeq'] = [video_seq] * len(df['Gyro X'])
  df['Time Window'] = up_win_l + low_win_l

If the video does not contain any outlier, use no_outlier_function instead of outlier function

In [ ]:
def no_outlier_function(df, x):
  a = df.loc[df['videoNum'] == x]
  a = a.reset_index(drop=True)
  return a

Two methods to check each video's head and end

In [ ]:
def head_end_1(df1, df2, df3, x):
  rounded_down = math.floor(min(df1['Recording timestamp'][0], df2['Recording timestamp'][0], df3['Recording timestamp'][0]) * 100) / 100
  rounded_up = math.ceil(max(df1['Recording timestamp'][len(df1['Recording timestamp']) - 1],\
                             df2['Recording timestamp'][len(df2['Recording timestamp']) - 1],\
                             df3['Recording timestamp'][len(df3['Recording timestamp']) - 1]) * 100) / 100
  return rounded_down, rounded_up

In [ ]:
def head_end_2(df1, df2, df3, x):
  rounded_down = math.floor(min(df1['Recording timestamp'][0], df2['Recording timestamp'][0], df3['Recording timestamp'][0]) * 100) / 100
  rounded_up = math.ceil(max(df1['Recording timestamp'][len(df1['Recording timestamp']) - 1],\
                             df2['Recording timestamp'][len(df2['Recording timestamp']) - 1],\
                             df3['Recording timestamp'][len(df3['Recording timestamp']) - 1]) * 100) / 100 + x
  return rounded_down, rounded_up

##Conduct window Length approximately 0.04s by using time anchor

###Video 1

In [ ]:
dfe_1_v1, dfe_2_v1 = outlier_function(df_e, 1)
dfg_1_v1, dfg_2_v1 = outlier_function(df_g, 1)
dfa_1_v1, dfa_2_v1 = outlier_function(df_a, 1)
a, b = head_end_1(dfe_1_v1, dfg_1_v1, dfa_1_v1, 0.04)
upper_window_list_v1 = win_list(a, b, 0.04)
a, b = head_end_2(dfe_2_v1, dfg_2_v1, dfa_2_v1, 0.04)
lower_window_list_v1 = win_list(a, b, 0.04)
df_v1_upper = output_data(dfg_1_v1, dfe_1_v1, dfa_1_v1, upper_window_list_v1)
df_v1_lower = output_data(dfg_2_v1, dfe_2_v1, dfa_2_v1, lower_window_list_v1)
df_v1 = pd.concat([df_v1_upper, df_v1_lower])
df_v1 = df_v1.reset_index(drop=True)
other_columns(df_v1, 3, 1, 1, upper_window_list_v1, lower_window_list_v1)

###Video 2

In [ ]:
dfe_1_v2, dfe_2_v2 = outlier_function(df_e, 2)
dfg_1_v2, dfg_2_v2 = outlier_function(df_g, 2)
dfa_1_v2, dfa_2_v2 = outlier_function(df_a, 2)
a, b = head_end_2(dfe_1_v2, dfg_1_v2, dfa_1_v2, 0.05)
upper_window_list_v2 = win_list(a, b, 0.05)
a, b = head_end_2(dfe_2_v2, dfg_2_v2, dfa_2_v2, 0.04)
lower_window_list_v2 = win_list(a, b, 0.04)
df_v2_upper = output_data(dfg_1_v2, dfe_1_v2, dfa_1_v2, upper_window_list_v2)
df_v2_lower = output_data(dfg_2_v2, dfe_2_v2, dfa_2_v2, lower_window_list_v2)
df_v2 = pd.concat([df_v2_upper, df_v2_lower])
df_v2 = df_v2.reset_index(drop=True)
other_columns(df_v2, 3, 2, 1, upper_window_list_v2, lower_window_list_v2)
merged_df = pd.concat([df_v1, df_v2])
merged_df = merged_df.reset_index(drop=True)

###Video 3

In [ ]:
dfe_1_v3, dfe_2_v3 = outlier_function(df_e, 3)
dfg_1_v3, dfg_2_v3 = outlier_function(df_g, 3)
dfa_1_v3, dfa_2_v3 = outlier_function(df_a, 3)
a, b = head_end_2(dfe_1_v3, dfg_1_v3, dfa_1_v3, 0.05)
upper_window_list_v3 = win_list(a, b, 0.05)
df_v3_upper = output_data(dfg_1_v3, dfe_1_v3, dfa_1_v3, upper_window_list_v3)
a, b = head_end_2(dfe_2_v3, dfg_2_v3, dfa_2_v3, 0.04)
lower_window_list_v3 = win_list(a, b, 0.04)
df_v3_lower = output_data(dfg_2_v3, dfe_2_v3, dfa_2_v3, lower_window_list_v3)
df_v3 = pd.concat([df_v3_upper, df_v3_lower])
df_v3 = df_v3.reset_index(drop=True)
other_columns(df_v3, 3, 3, 1, upper_window_list_v3, lower_window_list_v3)
merged_df = pd.concat([merged_df, df_v3])
merged_df = merged_df.reset_index(drop=True)

###Video 4

In [ ]:
dfe_1_v4, dfe_2_v4 = outlier_function(df_e, 4)
dfg_1_v4, dfg_2_v4 = outlier_function(df_g, 4)
dfa_1_v4, dfa_2_v4 = outlier_function(df_a, 4)
a, b = head_end_2(dfe_1_v4, dfg_1_v4, dfa_1_v4, 0.04)
upper_window_list_v4 = win_list(a, b, 0.04)
a, b = head_end_2(dfe_2_v4, dfg_2_v4, dfa_2_v4, 0.04)
lower_window_list_v4 = win_list(a, b, 0.04)
df_v4_upper = output_data(dfg_1_v4, dfe_1_v4, dfa_1_v4, upper_window_list_v4)
df_v4_lower = output_data(dfg_2_v4, dfe_2_v4, dfa_2_v4, lower_window_list_v4)
df_v4 = pd.concat([df_v4_upper, df_v4_lower])
df_v4 = df_v4.reset_index(drop=True)
other_columns(df_v4, 3, 4, 1, upper_window_list_v4, lower_window_list_v4)
merged_df = pd.concat([merged_df, df_v4])
merged_df = merged_df.reset_index(drop=True)

###Video 5

In [ ]:
dfe_1_v5 = no_outlier_function(df_e, 5)
dfg_1_v5 = no_outlier_function(df_g, 5)
dfa_1_v5 = no_outlier_function(df_a, 5)
a, b = head_end_2(dfe_1_v5, dfg_1_v5, dfa_1_v5, 0.04)
upper_window_list_v5 = win_list(a, b, 0.04)
df_v5_upper = output_data(dfg_1_v5, dfe_1_v5, dfa_1_v5, upper_window_list_v5)
df_v5_upper['order'] = [3] * len(df_v5_upper['Gyro X'])
df_v5_upper['videoNum'] = [5] * len(df_v5_upper['Gyro X'])
df_v5_upper['videoSeq'] = [1] * len(df_v5_upper['Gyro X'])
df_v5_upper['Time Window'] = upper_window_list_v5
merged_df = pd.concat([merged_df, df_v5_upper])
merged_df = merged_df.reset_index(drop=True)


###Video 6

In [ ]:
dfe_1_v6 = no_outlier_function(df_e, 6)
dfg_1_v6 = no_outlier_function(df_g, 6)
dfa_1_v6 = no_outlier_function(df_a, 6)
a, b = head_end_2(dfe_1_v6, dfg_1_v6, dfa_1_v6, 0.04)
upper_window_list_v6 = win_list(a, b, 0.04)
df_v6_upper = output_data(dfg_1_v6, dfe_1_v6, dfa_1_v6, upper_window_list_v6)
df_v6_upper['order'] = [1] * len(df_v6_upper['Gyro X'])
df_v6_upper['videoNum'] = [6] * len(df_v6_upper['Gyro X'])
df_v6_upper['videoSeq'] = [1] * len(df_v6_upper['Gyro X'])
df_v6_upper['Time Window'] = upper_window_list_v6
merged_df = pd.concat([merged_df, df_v6_upper])
merged_df = merged_df.reset_index(drop=True)

###Video 7

In [ ]:
dfe_1_v7 = no_outlier_function(df_e, 7)
dfg_1_v7 = no_outlier_function(df_g, 7)
dfa_1_v7 = no_outlier_function(df_a, 7)
a, b = head_end_2(dfe_1_v7, dfg_1_v7, dfa_1_v7, 0.04)
upper_window_list_v7 = win_list(a, b, 0.04)
df_v7_upper = output_data(dfg_1_v7, dfe_1_v7, dfa_1_v7, upper_window_list_v7)
df_v7_upper['order'] = [3] * len(df_v7_upper['Gyro X'])
df_v7_upper['videoNum'] = [7] * len(df_v7_upper['Gyro X'])
df_v7_upper['videoSeq'] = [1] * len(df_v7_upper['Gyro X'])
df_v7_upper['Time Window'] = upper_window_list_v7
merged_df = pd.concat([merged_df, df_v7_upper])
merged_df = merged_df.reset_index(drop=True)


###Video 8

In [ ]:
dfe_1_v8, dfe_2_v8 = outlier_function(df_e, 8)
dfg_1_v8, dfg_2_v8 = outlier_function(df_g, 8)
dfa_1_v8, dfa_2_v8 = outlier_function(df_a, 8)
a, b = head_end_2(dfe_1_v8, dfg_1_v8, dfa_1_v8, 0.04)
upper_window_list_v8 = win_list(a, b, 0.04)
a, b = head_end_2(dfe_2_v8, dfg_2_v8, dfa_2_v8, 0.04)
lower_window_list_v8 = win_list(a, b, 0.04)
df_v8_upper = output_data(dfg_1_v8, dfe_1_v8, dfa_1_v8, upper_window_list_v8)
df_v8_lower = output_data(dfg_2_v8, dfe_2_v8, dfa_2_v8, lower_window_list_v8)
df_v8 = pd.concat([df_v8_upper, df_v8_lower])
df_v8 = df_v8.reset_index(drop=True)
other_columns(df_v8, 2, 8, 2, upper_window_list_v8, lower_window_list_v8)
merged_df = pd.concat([merged_df, df_v8])
merged_df = merged_df.reset_index(drop=True)

###Video 9

In [ ]:
dfe_1_v9, dfe_2_v9 = outlier_function(df_e, 9)
dfg_1_v9, dfg_2_v9 = outlier_function(df_g, 9)
dfa_1_v9, dfa_2_v9 = outlier_function(df_a, 9)
a, b = head_end_2(dfe_1_v9, dfg_1_v9, dfa_1_v9, 0.04)
upper_window_list_v9 = win_list(a, b, 0.04)
a, b = head_end_2(dfe_2_v9, dfg_2_v9, dfa_2_v9, 0.05)
lower_window_list_v9 = win_list(a, b, 0.05)
df_v9_upper = output_data(dfg_1_v9, dfe_1_v9, dfa_1_v9, upper_window_list_v9)
df_v9_lower = output_data(dfg_2_v9, dfe_2_v9, dfa_2_v9, lower_window_list_v9)
df_v9 = pd.concat([df_v9_upper, df_v9_lower])
df_v9 = df_v9.reset_index(drop=True)
other_columns(df_v9, 2, 9, 2, upper_window_list_v9, lower_window_list_v9)
merged_df = pd.concat([merged_df, df_v9])
merged_df = merged_df.reset_index(drop=True)

###Video 10

In [ ]:
dfe_1_v10 = no_outlier_function(df_e, 10)
dfg_1_v10 = no_outlier_function(df_g, 10)
dfa_1_v10 = no_outlier_function(df_a, 10)
a, b = head_end_2(dfe_1_v10, dfg_1_v10, dfa_1_v10, 0.04)
upper_window_list_v10 = win_list(a, b, 0.04)
df_v10_upper = output_data(dfg_1_v10, dfe_1_v10, dfa_1_v10, upper_window_list_v10)
df_v10_upper['order'] = [2] * len(df_v10_upper['Gyro X'])
df_v10_upper['videoNum'] = [10] * len(df_v10_upper['Gyro X'])
df_v10_upper['videoSeq'] = [2] * len(df_v10_upper['Gyro X'])
df_v10_upper['Time Window'] = upper_window_list_v10
merged_df = pd.concat([merged_df, df_v10_upper])
merged_df = merged_df.reset_index(drop=True)

##Save as CSV file for later feature engineering

In [ ]:
merged_df.to_csv('your_data.csv', index=False)